In [ ]:
#
# Started with voronoi_6.ipynb...
#
import polars as pl
import numpy as np
import networkx as nx
import random
import rtsvg
rt = rtsvg.RACETrack()

def randomCircleConfig(n_circles=15, w=128, h=128, r_min=3.0, r_max=15.0, d_intra_circle_min=3.0):
    attempts, circles = 0, []
    while len(circles) < n_circles:
        r = random.uniform(r_min, r_max)
        x = random.uniform(r+d_intra_circle_min, w-r-d_intra_circle_min)
        y = random.uniform(r+d_intra_circle_min, h-r-d_intra_circle_min)
        overlaps = False
        for c in circles:
            d = rt.segmentLength(((x,y), c))
            if c[2] + r + d_intra_circle_min > d : overlaps = True
        if overlaps == False: circles.append((x,y,r))
        attempts += 1
        if attempts > 1000: raise Exception('Failed to generate circles')
    return circles

circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
circles = [(85,  53, 12), (77, 111,  5), (24,  58, 13), (19,  19, 13),
           (59,  21,  9), (93,  16, 10), (83,  79,  6), (16, 106,  5)]
circles = randomCircleConfig()
svg_hdr     = '<svg x="0" y="0" width="768" height="768" viewbox="-2 -2 132 132"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'
svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.4"/>')
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="0.5" fill="{rt.co_mgr.getColor(i)}"/>')
#rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])

In [170]:
# return bisector of points p0 and p1 -- bisector is ((x,y),(u,v)) where u,v is the vector of the bisector
def _isedgar_bisector(p0, p1):
    x, y     = (p0[0] + p1[0])/2.0, (p0[1] + p1[1])/2.0
    uv       = rt.unitVector((p0, p1))
    pdx, pdy = uv[1], -uv[0]
    return ((x,y), (pdx,pdy))
# For the circle version
def _isedgar_bisectorForCircles(c0, c1):
    # Let's make sure that the bisector is always done the save way irrespective of the circle order
    if   c0[0] >  c1[0]:                   c0, c1 = c1, c0
    elif c0[0] == c1[0] and c0[1] > c1[1]: c0, c1 = c1, c0
    # Unit vector between the two
    uv              = rt.unitVector((c0, c1))
    x0, y0          = c0[0] + uv[0] * c0[2], c0[1] + uv[1] * c0[2]
    x1, y1          = c1[0] - uv[0] * c1[2], c1[1] - uv[1] * c1[2]
    x,  y           = (x0+x1)/2.0, (y0+y1)/2.0
    pdx, pdy        = uv[1], -uv[0]
    return ((x,y), (pdx,pdy))
# returns vertices that intersect the polygon
def _isedgar_intersects(bisects, poly, merge_threshold):
    inters, already_added = [], set()
    for i in range(0, len(poly)):
        p0, p1 = poly[i], poly[(i+1)%len(poly)]
        xy = rt.lineSegmentIntersectionPoint((bisects[0], (bisects[0][0] + bisects[1][0], bisects[0][1] + bisects[1][1])),(p0, p1))
        if xy is not None:
            too_close = False
            for pt in already_added:
                if rt.segmentLength((pt, xy)) < merge_threshold:
                    too_close = True
                    break
            if too_close == False:
                inters.append((xy, i, (i+1)%len(poly)))
                already_added.add(xy)
    return inters
# conctenate a point, a list, and a point into a new list
def _isedgar_createCell(my_cell, p0, p1, i0, i1, sign):
    l = [p0]
    i = i0
    while i != i1:
        l.append(my_cell[i])
        i += sign
        if i < 0:             i += len(my_cell)
        if i >= len(my_cell): i -= len(my_cell)
    l.append(my_cell[i1])
    l.append(p1)
    return l
# contain true if pt is in poly
def _isedgar_contains(poly, pt):
    inter_count = 0
    for i in range(len(poly)):
        p0, p1 = poly[i], poly[(i+1)%len(poly)]
        _tuple_ = rt.segmentsIntersect((pt,(pt[0]+1e9,pt[1])),(p0,p1)) # use a ray from the pt to test
        if _tuple_[0] and (_tuple_[1], _tuple_[2]) != p1: inter_count += 1
    if inter_count%2 == 1: return True
    return False

In [ ]:
#
# Algorithm source & description from:
# "A simple algorithm for 2D Voronoi diagrams"
# https://www.youtube.com/watch?v=I6Fen2Ac-1U
# https://gist.github.com/isedgar/d445248c9ff6c61cef44fc275cb2398f
#
def isedgarVoronoi(self, S, Box=None, pad=10, use_circle_radius=False, merge_threshold=0.5):
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells = []
    for i in range(len(S)):
        p                  = S[i]
        cell               = Box
        for q in S:
            if p == q: continue
            B            = _isedgar_bisector(p,q) if use_circle_radius == False else _isedgar_bisectorForCircles(p,q)
            B_intersects = _isedgar_intersects(B, cell, merge_threshold)
            if len(B_intersects) == 2:
                t1, t2  = B_intersects[0][0], B_intersects[1][0] # these are _xy_ tuples
                xi, xj  = B_intersects[0][2], B_intersects[1][1]
                newCell = _isedgar_createCell(cell, t1, t2, xi, xj, 1)
                if _isedgar_contains(newCell, p) == False:
                    xi, xj  = B_intersects[0][1], B_intersects[1][2]
                    newCell = _isedgar_createCell(cell, t1, t2, xi, xj, -1)
                cell = newCell
        cells.append(cell)
    return cells

polys = isedgarVoronoi(rt, circles, [(0,0),(0,128),(128,128),(128,0)], use_circle_radius=True)
voronoi_base_svg, xy_set, xy_to_circles, xy_to_segments = [], set(), {}, {}

def trackXYs(_xy_, _circle_, _segment_):
    if _xy_ not in xy_to_circles: 
        xy_to_circles[_xy_]  = set()
        xy_to_segments[_xy_] = set()
    if _segment_[0] not in xy_to_segments: xy_to_segments[_segment_[0]] = set()
    if _segment_[1] not in xy_to_segments: xy_to_segments[_segment_[1]] = set()
    xy_to_circles [_xy_].add(_circle_)
    xy_to_segments[_xy_].add(_segment_)
    xy_to_segments[_segment_[0]].add(_segment_)
    xy_to_segments[_segment_[1]].add(_segment_)

for i in range(len(circles)):
    _color_ = rt.co_mgr.getColor(i)
    _poly_  = polys[i]
    d       = f'M {_poly_[0][0]} {_poly_[0][1]} '
    trackXYs(_poly_[0], circles[i], (_poly_[-1], _poly_[0]))
    for j in range(1,len(_poly_)): 
        d += f'L {_poly_[j][0]} {_poly_[j][1]} '
        trackXYs(_poly_[j], circles[i], (_poly_[j-1], _poly_[j]))
    d += 'Z'
    voronoi_base_svg.append(f'<path d="{d}" fill="none" stroke="{_color_}" stroke-width="0.4"/>')
#for _xy_ in xy_to_circles:
#    voronoi_base_svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5+random.random()}" fill="none" stroke="black" stroke-width="0.1"/>')

segment_perc = 1.0/4.0
for _xy_ in xy_to_circles:
    _longest_segment_d_ = 0.0
    for _segment_ in xy_to_segments[_xy_]:
        _d_ = rt.segmentLength(_segment_)
        if _d_ > _longest_segment_d_: _longest_segment_d_ = _d_
    closest_circle_d = 1e9
    for _circle_ in xy_to_circles[_xy_]:
        _d_ = rt.segmentLength((_xy_, _circle_))
        if _d_ < closest_circle_d: closest_circle_d = _d_
    r = min(closest_circle_d - 3.0, _longest_segment_d_ * segment_perc, 4.0)
    voronoi_base_svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{r}" fill="none" stroke="black" stroke-width="0.1"/>')

rt.tile([svg_hdr + ''.join(voronoi_base_svg) + ''.join(svg_circles) + svg_ftr])

In [172]:
def randomCircleConfig(n_circles=15, w=128, h=128, r_min=3.0, r_max=15.0, d_intra_circle_min=3.0):
    circles = []
    while len(circles) < n_circles:
        r = random.uniform(r_min, r_max)
        x = random.uniform(r+d_intra_circle_min, w-r-d_intra_circle_min)
        y = random.uniform(r+d_intra_circle_min, h-r-d_intra_circle_min)
        overlaps = False
        for c in circles:
            d = rt.segmentLength(((x,y), c))
            if c[2] + r > d + d_intra_circle_min: overlaps = True
        if overlaps == False: circles.append((x,y,r))
    return circles

